<a href="https://colab.research.google.com/github/GIMMI42PIASTRATO/AI-Chess-BOT/blob/main/AI_Chess_BOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation of all the dependencies

In [1]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle

cp: cannot stat 'kaggle.json': No such file or directory


In [2]:
!kaggle datasets download arevel/chess-games
!unzip -qq /content/chess-games.zip

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
unzip:  cannot find or open /content/chess-games.zip, /content/chess-games.zip.zip or /content/chess-games.zip.ZIP.


In [3]:
!pip install chess -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.6 MB/s eta 0:00:00


#CODE

In [4]:
import chess
import numpy as np
import re

##Mapping the chess board

In [5]:
letter_to_num = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
num_to_letter = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h'}

##Chess board to matrix

###Board to rappresentation
This function create a numpy tensor (3D numpyarray) which rapresent the board containing every position of every chess pice.
Every chess pice has is own mapping (Is a matrix representing the chessboard that maps the position of only one type of pieces)

In [5]:
def board_to_rep(board: chess.Board()):
  pieces = ['p', 'r', 'n', 'b', 'q', 'k'] #Initials of the names of the pieces
  layers = []
  for piece in pieces:
    layers.append(layer_to_rep(board, piece))
  board_rep = np.stack(layers)
  return board_rep

###Layer rappresentation

In [6]:
def layer_to_rep(board, piece):
  str_board = str(board)
  str_board = re.sub(f"[^{piece}{piece.upper()} \n]", ".", str_board)
  str_board = re.sub(f"{piece}", "1", str_board)
  str_board = re.sub(f"{piece.upper()}", "-1", str_board)
  str_board = re.sub("\.", "0", str_board)

  board_matrix = []
  for row in str_board.split("\n"):
    row = row.split()
    row = [int(x) for x in row]
    board_matrix.append(row)

  return np.array(board_matrix)

##Chess move to matrix

###Move to rappresentation

In [8]:
def move_to_rep(move, board: chess.Board()):
  board.push_san(move).uci()
  move = str(board.pop())

  from_output_layer = np.zeros((8, 8))
  from_row = 8 - int(move[1])
  from_column = letter_to_num[move[0]]
  from_output_layer[from_row][from_column] = 1

  to_output_layer = np.zeros((8, 8))
  to_row = 8 - int(move[3])
  to_column = letter_to_num[move[2]]
  to_output_layer[to_row][to_column]

  return np.stack((from_output_layer, to_output_layer))